## PUMS Preprocessing

Download all PUMS person-level state files for a given year, extract relevant columns, concatenate the data for all states into a single USA dataframe, and write result file to disk.

In [1]:
import os
import re
import sys
import zipfile
import requests
import pandas as pd
from urllib.parse import urljoin

In [2]:
# year of PUMS data
YEAR = 2019

# URL of the FTP site for the PUMS 2020 1-year data files
if YEAR <= 2019:
    # these years have production weights
    PUMS_URL = 'https://www2.census.gov/programs-surveys/acs/data/pums/{0}/1-Year/'.format(YEAR)
else:
    # experimental weights because of the Covid pandemic
    PUMS_URL = 'https://www2.census.gov/programs-surveys/acs/experimental/{0}/data/pums/1-Year/'.format(YEAR)

# output file
OUTPUT_FILE = 'pums_usa_{0}.csv'.format(YEAR)

# These are the two-letter abbreviations used by the pums files.
# Each person-level filename has this form: "csv_pxy.zip", where
# 'xy' represents the two-letter abbreviation for a jurisdiction
# in this list:
JURISDICTIONS = [
    'ak', # Alaska
    'al', # Alabama
    'ar', # Arkansas
    'az', # Arizona
    'ca', # California
    'co', # Colorado
    'ct', # Connecticut
    'dc', # District of Columbia *** (PUMS includes DC also)
    'de', # Delaware
    'fl', # Florida
    'ga', # Georgia
    'hi', # Hawaii
    'ia', # Iowa
    'id', # Idaho
    'il', # Illinois
    'in', # Indiana
    'ks', # Kansas
    'ky', # Kentucky
    'la', # Louisiana
    'ma', # Massachusetts
    'md', # Maryland
    'me', # Maine
    'mi', # Michigan
    'mn', # Minnesota
    'mo', # Missouri
    'ms', # Mississippi
    'mt', # Montana
    'nc', # North Carolina
    'nd', # North Dakota
    'ne', # Nebraska
    'nh', # New Hampshire
    'nj', # New Jersey
    'nm', # New Mexico
    'nv', # Nevada
    'ny', # New York
    'oh', # Ohio
    'ok', # Oklahoma
    'or', # Oregon
    'pa', # Pennsylvania
    'ri', # Rhode Island
    'sc', # South Carolina
    'sd', # South Dakota
    'tn', # Tennessee
    'tx', # Texas
    'ut', # Utah
    'va', # Virginia
    'vt', # Vermont
    'wa', # Washington
    'wi', # Wisconsin
    'wv', # West Virginia
    'wy', # Wyoming
]

# should have all 50 states + DC
assert 51 == len(set(JURISDICTIONS))

# columns to extract from each file
KEEP_COLS = [
    'ST',       # state code based on 2010 census definitions
    'SERIALNO', # essentially the household ID
    'SPORDER',  # unique ID for each person within a given household
    'AGEP',     # numeric age, 0..99
    'ADJINC',   # income adjustment factor to constant dollars
    'PWGTP',    # person-level weight
    'HISP',     # recoded detailed Hispanic origin (01 == not Spanish/Hispanic/Latino)
    'RAC1P',    # detailed race code
    'SCHL',     # educational attainment
    'HICOV',    # health insurance recode
    'PINCP',    # total person's income (use ADJINC to adjust to constant dollars)
    'SEX',      # sex (at birth, presumably)
]

#### Download all zip files for all jurisdictions

In [3]:
# folder to hold all state files
STATE_ZIP_FOLDER = 'state_zip_files'
if not os.path.exists(STATE_ZIP_FOLDER):
    os.makedirs(STATE_ZIP_FOLDER)
    
num_downloaded = 0
for i,abbrev in enumerate(JURISDICTIONS):
    # source PUMS file
    pums_file = 'csv_p{0}.zip'.format(abbrev)
    # destination file
    output_file = os.path.join(STATE_ZIP_FOLDER, pums_file)
    with open(output_file, 'wb') as outfile:
        url = urljoin(PUMS_URL, pums_file)
        zip_data = requests.get(url, stream=True).content
        outfile.write(zip_data)
        print('Downloaded {0} ({1:2d}/{2})'.format(pums_file, i+1, len(JURISDICTIONS)))
        num_downloaded += 1
        
assert len(JURISDICTIONS) == num_downloaded

Downloaded csv_pak.zip ( 1/51)
Downloaded csv_pal.zip ( 2/51)
Downloaded csv_par.zip ( 3/51)
Downloaded csv_paz.zip ( 4/51)
Downloaded csv_pca.zip ( 5/51)
Downloaded csv_pco.zip ( 6/51)
Downloaded csv_pct.zip ( 7/51)
Downloaded csv_pdc.zip ( 8/51)
Downloaded csv_pde.zip ( 9/51)
Downloaded csv_pfl.zip (10/51)
Downloaded csv_pga.zip (11/51)
Downloaded csv_phi.zip (12/51)
Downloaded csv_pia.zip (13/51)
Downloaded csv_pid.zip (14/51)
Downloaded csv_pil.zip (15/51)
Downloaded csv_pin.zip (16/51)
Downloaded csv_pks.zip (17/51)
Downloaded csv_pky.zip (18/51)
Downloaded csv_pla.zip (19/51)
Downloaded csv_pma.zip (20/51)
Downloaded csv_pmd.zip (21/51)
Downloaded csv_pme.zip (22/51)
Downloaded csv_pmi.zip (23/51)
Downloaded csv_pmn.zip (24/51)
Downloaded csv_pmo.zip (25/51)
Downloaded csv_pms.zip (26/51)
Downloaded csv_pmt.zip (27/51)
Downloaded csv_pnc.zip (28/51)
Downloaded csv_pnd.zip (29/51)
Downloaded csv_pne.zip (30/51)
Downloaded csv_pnh.zip (31/51)
Downloaded csv_pnj.zip (32/51)
Download

#### Unzip and process each file

In [4]:
def count_unique_individuals(df):
    """
    Form an identifier from the SERIALNO and SPORDER fields. The SERIALNO field is a household or family ID,
    and the SPORDER field is the person ID within the household or family. The set of all combinations of
    these fields for a given state should be a unique identifier for the people of that state.
    """
    
    id_set = set()
    for index, row in df.iterrows():
        # household/family ID
        serialno = row['SERIALNO']
        # person id within the household
        person = row['SPORDER']
        key = '{0}|{1}'.format(serialno, person)
        id_set.add(key)
    unique_individuals = len(id_set)
    return unique_individuals

In [5]:
corrupted = []
dataframes = []

file_index = 0
for f in os.listdir(STATE_ZIP_FOLDER):
    filename = os.path.join(STATE_ZIP_FOLDER, f)
    if not filename.endswith('.zip'):
        continue
    print('Processing {0} ({1}/{2})'.format(f, file_index+1, num_downloaded))
    try:
        with zipfile.ZipFile(filename, 'r') as zf:
            members = zf.infolist()
            for m in members:
                member_filename = m.filename
                if member_filename.endswith('.pdf'):
                    # skip documentation
                    continue
                with zf.open(member_filename) as csvfile:
                    df = pd.read_csv(csvfile)
                    # extract desired columns
                    df = df[KEEP_COLS].copy()
                    # count unique individuals
                    uniques = count_unique_individuals(df)
                    # should have a single unique individual in each row of the dataframe
                    assert df.shape[0] == uniques
                    dataframes.append(df)
    except zipfile.BadZipFile:
        corrupted.append(filename)
    file_index += 1
        
print('\nProcessed {0} files.'.format(file_index))
num_corrupted = len(corrupted)
if 0 == num_corrupted:
    print('No corrupted zip files found.')
else:
    print('Found these corrupted zip files: {0}'.format(corrupted))

Processing csv_pin.zip (1/51)
Processing csv_phi.zip (2/51)
Processing csv_psc.zip (3/51)
Processing csv_pil.zip (4/51)
Processing csv_pks.zip (5/51)
Processing csv_psd.zip (6/51)
Processing csv_pga.zip (7/51)
Processing csv_pla.zip (8/51)
Processing csv_pme.zip (9/51)
Processing csv_put.zip (10/51)
Processing csv_pms.zip (11/51)
Processing csv_pnh.zip (12/51)
Processing csv_pmd.zip (13/51)
Processing csv_pco.zip (14/51)
Processing csv_par.zip (15/51)
Processing csv_pnj.zip (16/51)
Processing csv_pwi.zip (17/51)
Processing csv_pmt.zip (18/51)
Processing csv_pok.zip (19/51)
Processing csv_pny.zip (20/51)
Processing csv_poh.zip (21/51)
Processing csv_pwy.zip (22/51)
Processing csv_pma.zip (23/51)
Processing csv_pnm.zip (24/51)
Processing csv_ptn.zip (25/51)
Processing csv_ptx.zip (26/51)
Processing csv_por.zip (27/51)
Processing csv_pnv.zip (28/51)
Processing csv_pal.zip (29/51)
Processing csv_pnc.zip (30/51)
Processing csv_pmo.zip (31/51)
Processing csv_pwa.zip (32/51)
Processing csv_pw

In [6]:
# concatenate the dataframes
usa_df = pd.concat(dataframes)
usa_df

,ST,SERIALNO,SPORDER,AGEP,ADJINC,PWGTP,HISP,RAC1P,SCHL,HICOV,PINCP,SEX
0,18,2019GQ0000001,1,35,1010145,71,1,1,17.0,2,63000.0,1
1,18,2019GQ0000007,1,21,1010145,24,1,6,19.0,1,2000.0,2
2,18,2019GQ0000033,1,21,1010145,15,1,1,19.0,1,7400.0,2
3,18,2019GQ0000125,1,19,1010145,43,1,1,15.0,2,0.0,1
4,18,2019GQ0000276,1,18,1010145,87,1,1,18.0,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
205289,12,2019HU1412413,4,6,1010145,76,1,1,4.0,1,NaN,1
205290,12,2019HU1412413,5,3,1010145,74,1,1,1.0,1,NaN,1
205291,12,2019HU1412447,1,59,1010145,56,1,1,22.0,1,5000.0,1
205292,12,2019HU1412447,2,57,1010145,57,1,1,20.0,2,130000.0,2


In [7]:
# sort by state
usa_df = usa_df.sort_values(by=['ST'])
usa_df

,ST,SERIALNO,SPORDER,AGEP,ADJINC,PWGTP,HISP,RAC1P,SCHL,HICOV,PINCP,SEX
43847,1,2019HU1259536,4,10,1010145,185,1,2,7.0,1,NaN,1
32613,1,2019HU0915454,2,17,1010145,77,1,1,13.0,1,10000.0,2
32614,1,2019HU0915504,1,46,1010145,23,2,1,19.0,1,94600.0,1
32615,1,2019HU0915555,1,69,1010145,71,1,1,16.0,1,37100.0,2
32616,1,2019HU0915555,2,72,1010145,53,1,1,21.0,1,8400.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4223,56,2019HU1000576,2,56,1010145,134,1,1,18.0,1,0.0,2
4224,56,2019HU1000576,3,17,1010145,141,1,1,14.0,1,0.0,1
4225,56,2019HU1000791,1,66,1010145,24,1,1,16.0,1,13900.0,2
4217,56,2019HU0999472,1,33,1010145,48,2,1,21.0,1,33700.0,2


In [8]:
print('Number of unique ST values: {0}'.format(len(set(usa_df['ST'].values))))

Number of unique ST values: 51


#### Write to disk

In [9]:
usa_df.to_csv(OUTPUT_FILE, index=False)
print('Wrote file "{0}".'.format(OUTPUT_FILE))

Wrote file "pums_usa_2019.csv".
